In [1]:
# Script to get rewards from the human demos

In [3]:
import datetime
import glob
import os
import hydra
import numpy as np
import torch
import torch.nn.functional as F
import sys

from pathlib import Path

from torchvision import transforms as T

from tactile_learning.models import *
from tactile_learning.utils import *
from tactile_learning.tactile_data import *
from holobot.robot.allegro.allegro_kdl import AllegroKDL

In [6]:
# Load the human demo
def load_human_demo(data_path, demo_id, view_num, image_transform=None): # This will have the exact same demo
    human_demos = []
    image_obs = [] 
    old_demo_id = -1

    if image_transform is None: 
        def viewed_crop_transform(image):
            return crop_transform(image, camera_view=view_num)
        image_transform = T.Compose([
            T.Resize((480,640)),
            T.Lambda(viewed_crop_transform),
            T.Resize(480),
            T.ToTensor(),
            T.Normalize(VISION_IMAGE_MEANS, VISION_IMAGE_STDS), 
        ])

    roots = glob.glob(f'{data_path}/demonstration_*')
    roots = sorted(roots)
    image_root = roots[demo_id]

    image_ids = glob.glob(f'{image_root}/cam_{view_num}_rgb_images/*')
    image_ids = sorted([int(image_id.split('_')[-1].split('.')[0]) for image_id in image_ids])
    print('image_ids: {}'.format(image_ids))
    for image_id in image_ids:
        image = load_dataset_image( # We only have images in human demonstrations
            data_path = data_path, 
            demo_id = demo_id, 
            image_id = image_id,
            view_num = view_num,
            transform = image_transform
        )
        image_obs.append(image)

    human_demos.append(dict(
        image_obs = torch.stack(image_obs, 0), 
    ))

    return human_demos

    # for step_id in range(len(data['image']['indices'])): 
    #     demo_id, _ = data['tactile']['indices'][step_id]
    #     if (demo_id != old_demo_id and step_id > 0) or (step_id == len(data['image']['indices'])-1): # NOTE: We are losing the last frame of the last expert
    #         expert_demos.append(dict(
    #             image_obs = torch.stack(image_obs, 0), 
    #         ))
    #         image_obs = [] 

    #     _, image_id = data['image']['indices'][step_id]
    #     image = load_dataset_image( # We only have images in human demonstrations
    #         data_path = data_path, 
    #         demo_id = demo_id, 
    #         image_id = image_id,
    #         view_num = view_num,
    #         transform = image_transform
    #     )
    #     image_obs.append(image)


    #     old_demo_id = demo_id

    # return expert_demos

In [7]:
human_demo = load_human_demo(
    data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/human_bowl_picking',
    demo_id = 1,
    view_num = 1
)

image_ids: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
# Get the episode demos
from tactile_learning.utils import load_all_episodes, load_episode_demos
all_episodes = load_all_episodes(
    roots = cfg.episode_roots, # Either one of them will be none
    root_path = cfg.episode_root_path
)
episode_demos = load_episode_demos(
    all_episodes= all_episodes,
    image_transform = T.Compose([
        T.ToTensor(),
        T.Normalize(VISION_IMAGE_MEANS, VISION_IMAGE_STDS)
    ])
)